In [8]:
import os
print(os.getcwd())
print('Inicio del Jupyter File.')

c:\Users\andre\OneDrive\Programación-Datos\Proyectos para portfolio-ideas\ADR Index para acciones argentinas
Inicio del Jupyter File.


# ArgDR Index para monitorear el interés internacional en la economía argentina

*Por Andrés Fernández Apenburg*

*Licenciado en Economía, Universidad de Buenos Aires*

## Introducción
Existe una clase de activos financieros llamados ADR (del inglés American Drawing Right). Un ADR es un certificado de titularidad, por parte de un residente estadounidense, de una acción de una empresa cuyas acciones cotizan en un mercado fuera de EEUU. En el caso argentino, se trata de certificados de titularidad de acciones que cotizan en la Bolsa de Comercio de Buenos Aires (BCBA). La compraventa de los ADR argentinos tiene lugar en la Bolsa de Comercio de Nueva York (NYSE) y los valores se expresan en dólares estadounidenses. Existen ADR para múltiples empresas importantes de Argentina, como YPF, Grupo Galicia, e IRSA. 

La cotización de los ADR argentinos es seguida atentamente por los medios de comunicación financieros del país y el exterior, ya que provee una valiosa señal de las expectativas futuras de generación de dólares por parte del sector privado argentino. Sin embargo, no se cuenta con un índice que permita sintetizar el conjunto de los precios, es decir, una única variable numérica, cuya variación día a día refleje la tendencia general de los precios de los múltiples ADR argentinos. 


## La solución propuesta


### Qué buscamos en un índice
Las propiedades deseables de un índice incluyen:
1. Que varíe su valor si varía el estado del fenómeno subyacente a medir (en este caso, las tendencias *generales* del valor de los ADR en su conjunto)
2. Que no se vea afectado por *outliers* (en este caso, firmas pequeñas que aumenten su valor accionario en proporciones mucho mayores que la tendencia general en el resto de las firmas)



### Definición del índice
El Índice ArgDR está definido como un promedio ponderado de todos los precios de ADR argentinos, donde los ponderadores sugeridos son la capitalización de mercado de cada firma como porcentaje de la capitalización bursátil colectiva del total de las firmas incluidas. 
Formalmente, dada una cantidad $N$ de ADR argentinos cotizando en el instante o periodo $t$ (por ejemplo, el inicio o cierre de la jornada, o las 12 del mediodía, o un cierto período sobre el cual se toman valores promedio para todos los activos), el índice queda definido como: 

$$ 
ArgDR_{t} = \sum_{i=1}^{N} A_{i, t} \cdot P_{i, t}
$$

Donde $A_{i,t}$ es la cotización en dólares del ADR $i$ en $t$, y $P_{i,t}$ es el ponderador del ADR $i$ en la jornada $t$, definido como:

$$
P_{i, t}=\frac{C_{i, t}}{\sum_{i}^{N}C_{i, t}}
$$

Donde ${C_{i, t}}$ es la capitalización bursátil de la empresa $i$, es decir, su número total de acciones multiplicado por $A_{i, t}$.

Bajo esta definición se cumple que $\sum_{i}^{N}P_{i,t} = 1$, para todo $t$, por lo que se trata de un conjunto válido de ponderadores.

En la práctica, el ponderador se puede tomar como fijo a lo largo de múltiples $t$ (índice tipo Laspeyres), utilizando, por ejemplo, las capitalizaciones bursátiles al cierre del trimestre anterior para calcular los ponderadores, de forma tal que resulta opcional su actualización diaria. 


### Por qué nuestro índice satisface las propiedades deseadas
1. Al tratarse de un promedio ponderado de precios de acciones individuales, si suben todas las acciones, necesariamente debe subir el valor del índice
2. Si una firma es más relevante en el mercado (medido en términos de la capitalización bursátil, esto es, del valor implícito de la compañía en función de la cantidad de acciones y su precio), el índice responderá con un mayor cambio porcentual ante la misma variación porcentual en el valor de dicho activo en comparación con una firma menos relevante, porque el valor de su ponderador será mayor.

# Obtención de los datos

## Web scraping desde investing.com


Descargamos un archivo CSV con la tabla que tiene investing.com mostrando las cotizaciones de ADR argentinos, el cual se puede descargar desde [esta](https://www.investing.com/equities/argentina-adrs) web.

Decidimos remover MercadoLibre de nuestro índice, ya que no es un ADR propiamente dicho (no cotiza en la BCBA salvo como CEDEAR) y sus resultados financieros dependen principalmente de sus actividades fuera de la Argentina, factores que las distinguen de todos los demás activos considerados.

In [9]:
import pandas as pd


#Cargamos distintas fuentes de datos para evaluar robustez

ADRs_20240102 = 'Data\Arg_ADRs_2024_01_02.csv'
ADRs_20240221 = 'Data\Arg_ADRs_2024_02_21.csv'
ADRs_20240223 = 'Data\Arg_ADRs_2024_02_23.csv'
latest = 'Data\Argentina ADRs.csv'


def generar_tabla(csvfile):
    
    import pandas as pd

    tablaADR = pd.read_csv(csvfile)

    tablaADR = pd.DataFrame(tablaADR)

    tablaADR.set_index('Name', inplace = True)

    #print(tablaADR.index.values)

    # Removemos MercadoLibre (MELI) de la tabla y creamos una 'tablaADR_sinMELI'

    tablaADR_sinMELI = tablaADR.drop(['MercadoLibre'])

    #print(tablaADR_sinMELI.index.values)

    tablaADR_sinMELI = tablaADR_sinMELI.sort_values(by ='Name')

    
    

    return tablaADR_sinMELI

generar_tabla(latest)


,Last,High,Low,Chg.,Chg. %,Vol.,Time
Name,,,,,,,
BBVA Argentina,6.330,6.510,6.190,0.140,2.26%,722.77K,15:59:59
Banco Macro B ADR,36.95,37.36,35.90,0.720,1.99%,288.09K,16:00:00
Bioceres Crop,13.09,13.19,13.00,-0.010,-0.08%,33.6K,15:59:59
Central Puerto,8.320,8.650,8.240,-0.160,-1.89%,160.25K,16:00:00
Cresud SACIF,8.280,8.373,8.010,0.070,0.85%,243.37K,16:00:01
Despegar.com,8.71,8.84,8.64,0.010,0.11%,122.94K,16:00:00
Edenor ADR,18.180,19.340,17.950,-0.790,-4.16%,167.6K,16:00:00
Grupo Financiero Galicia ADR,22.680,22.970,21.860,0.725,3.30%,1.03M,16:00:01
Grupo Supervielle,4.420,4.655,4.360,0.050,1.14%,1.4M,16:00:00


## Cálculo de los ponderadores

Obtenemos la capitalización bursátil de cada firma de la web [https://www.nasdaq.com/market-activity/stocks](https://www.nasdaq.com/market-activity/stocks) al 21 de febrero del 2024. El programa es lo suficientemente flexible para admitir la modificación de los ponderadores en una fecha futura, si la trayectoria de las capitalizaciones bursátiles hace variar la importancia relativa de cada firma.

In [15]:
cap_bursatiles_feb24 = {
    'YPF Sociedad Anonima' : 6669706272,
    'Grupo Supervielle' : 379972012,
    'Grupo Financiero Galicia ADR' : 3064410163,
    'BBVA Argentina' : 1169255067,
    'Banco Macro B ADR' : 2153544365,
    'Telecom Argentina ADR' : 3303757407,
    'Cresud SACIF' : 495623283,
    'Central Puerto' : 1268750654,
    'Pampa Energia ADR' : 2326836834,
    'Loma Negra ADR' : 757609060,
    'Despegar.com' : 564813000,
    'IRSA ADR' : 564813000,
    'Transportadora Gas ADR' : 1981267110,
    'Bioceres Crop' : 806938546,
    'Edenor ADR' : 858866207}

cap_bursatiles = cap_bursatiles_feb24

suma_cap = sum(cap_bursatiles.values())

print(f'Las capitalizaciones suman {suma_cap}')

firmas_y_pond = []

for i in cap_bursatiles:
    firmas_y_pond.append([i, (cap_bursatiles[i]/suma_cap)])

ponderadores = dict(firmas_y_pond)


print()
print()
print()

print('PONDERADORES:')
for i in ponderadores:
    print(i, ':', round(ponderadores[i], 3))


print()
print()


print(f'Los ponderadores suman {round(sum(ponderadores.values()), 2)}, como corresponde.')



Las capitalizaciones suman 26366162980



PONDERADORES:
YPF Sociedad Anonima : 0.253
Grupo Supervielle : 0.014
Grupo Financiero Galicia ADR : 0.116
BBVA Argentina : 0.044
Banco Macro B ADR : 0.082
Telecom Argentina ADR : 0.125
Cresud SACIF : 0.019
Central Puerto : 0.048
Pampa Energia ADR : 0.088
Loma Negra ADR : 0.029
Despegar.com : 0.021
IRSA ADR : 0.021
Transportadora Gas ADR : 0.075
Bioceres Crop : 0.031
Edenor ADR : 0.033


Los ponderadores suman 1.0, como corresponde.


# Cómputo del índice

Teniendo los datos de precios y los ponderadores, podemos calcular el índice ArgDR para el 21-02-2024 (ya que tenemos los datos de precios para esa fecha y los ponderadores que se reutilizan de jornada en jornada, actualizándose únicamente a mediano plazo)

In [11]:
def preciosypond(csvfile):
    generar_tabla(csvfile)
    ponderadores_df = pd.DataFrame(ponderadores.values(), index=ponderadores.keys())
    ponderadores_df = ponderadores_df.sort_index()
    #print(ponderadores_df)
    datos_df = generar_tabla(csvfile)
    precios_y_pond = ponderadores_df.merge(datos_df, left_on=ponderadores_df.index, right_on = 'Name')
    precios_y_pond.rename(columns = {0 : 'Ponderador'}, inplace=True)
    precios_y_pond.to_csv('Data\preciosponddf.csv')
    return precios_y_pond

'''
Podemos ver que, al fluctuar los precios de las acciones, los datos provenientes 
 del primer archivo difieren de los del segundo
'''


preciosypond(latest).sort_values(by='Ponderador', ascending = False)


,Name,Ponderador,Last,High,Low,Chg.,Chg. %,Vol.,Time
14,YPF Sociedad Anonima,0.252965,17.905,18.255,17.760,-0.135,-0.75%,1.95M,16:00:00
12,Telecom Argentina ADR,0.125303,7.880,8.000,7.730,0.050,0.64%,188.6K,15:59:59
7,Grupo Financiero Galicia ADR,0.116225,22.680,22.970,21.860,0.725,3.30%,1.03M,16:00:01
11,Pampa Energia ADR,0.088251,43.41,45.54,43.28,-1.130,-2.54%,142.63K,16:00:00
1,Banco Macro B ADR,0.081678,36.95,37.36,35.90,0.720,1.99%,288.09K,16:00:00
13,Transportadora Gas ADR,0.075144,14.130,14.600,13.838,-0.260,-1.81%,112.02K,16:00:00
3,Central Puerto,0.048120,8.320,8.650,8.240,-0.160,-1.89%,160.25K,16:00:00
0,BBVA Argentina,0.044347,6.330,6.510,6.190,0.140,2.26%,722.77K,15:59:59
6,Edenor ADR,0.032575,18.180,19.340,17.950,-0.790,-4.16%,167.6K,16:00:00
2,Bioceres Crop,0.030605,13.09,13.19,13.00,-0.010,-0.08%,33.6K,15:59:59


In [12]:
from datetime import datetime
import os
import time

def fechacsv(csvfile) :
    creationdate = str(datetime.fromtimestamp(os.path.getctime(csvfile)))
    return creationdate[0:16]

print(fechacsv(latest))

2024-02-26 18:54


In [13]:





def calculo_indice(col, csvfile):
    precios_y_pond = preciosypond(csvfile)
    vector_pond = pd.Series(precios_y_pond['Ponderador'])
    #print(vector_pond)
    vector_ult_precio = pd.Series(precios_y_pond[col], dtype='float')
    #print(vector_ult_precio)
    ArgDR_Index = vector_pond.dot(vector_ult_precio)
    #print(f'El índice ArgDR ({col}) el día {fechacsv(csvfile)} vale {round(ArgDR_Index, 3)} puntos')
    return ArgDR_Index




#calculo_indice('Last', ADRs_20240221)
#calculo_indice('Low',  ADRs_20240221)
#calculo_indice('High', ADRs_20240221)
#print(generar_tabla(ADRs_20240223))
#calculo_indice('Last', ADRs_20240223)



adr20240102 = calculo_indice('Last', ADRs_20240102)
adr20240221 = calculo_indice('Last', ADRs_20240221)
adr20240223 = calculo_indice('Last', ADRs_20240223)

def printrnd3(float):
    print(round((float), 3))

print(adr20240223)
#printrnd3(adr20240223)
#printrnd3(adr20240224)
printrnd3(calculo_indice('Last',latest))

18.597431841402205
18.514


# Serie histórica

Creamos un archivo CSV denominado 'Serie_Historica_ArgDR', y una función para agregarle registros con el valor computado del índice en la fecha correspondiente. De esta forma, almacenamos una serie de tiempo en un CSV, lo cual nos permite guardar información histórica sobre la evolución del índice a lo largo del tiempo.

In [14]:
def registrar(csvfile):
    registro = {0:[fechacsv(csvfile)], 1: [calculo_indice('Last', csvfile)]}
    registro = pd.DataFrame(data=registro)
    print(registro)
    registro.to_csv('Data\Serie_Historica_ArgDR.csv', mode='a', header=False, index=False)

registrar(latest)



                  0        1
0  2024-02-26 18:54  18.5144
